# 6. HCC1806 Predictions
We will now use the Ensemble method developed in the training notebook in order to predict the environment of the cells in the test dataset.

In [2]:
import pandas as pd
import pickle
import numpy as np

# 1. Importing the files

In [3]:
tumor = 'HCC1806'

In [4]:
df_smartseq_pathHFiltNorm = f'../ai_lab/SmartSeq/{tumor}_SmartS_Filtered_Normalised_3000_Data_test_anonim.txt'
df_smartseq_HFiltNorm = pd.read_csv(df_smartseq_pathHFiltNorm, delimiter=' ')

In [5]:
df_dropseq_pathHFiltNorm = f'../ai_lab/DropSeq/{tumor}_Filtered_Normalised_3000_Data_test_anonim.txt'
df_dropseq_HFiltNorm = pd.read_csv(df_dropseq_pathHFiltNorm, delimiter=' ')

We take a look at the structure of this dataset

In [6]:
df_smartseq_HFiltNorm.head()

,1,2,3,4,5,6,7,8,9,10,...,36,37,38,39,40,41,42,43,44,45
DDIT4,0,2475,9088,6909,13655,7684,13038,640,3,542,...,4968,301,2264,122,1845,3654,437,10496,1506,1035
ANGPTL4,0,0,2143,3086,2196,1619,1917,1,1,81,...,1634,153,127,47,643,1807,2,0,374,0
CALML5,0,0,0,0,0,6,0,0,0,0,...,797,0,0,0,0,0,1526,1,635,0
KRT14,169,0,0,909,1,74,108,1,1794,99,...,53,745,0,1917,1,0,3143,643,349,998
CCNB1,233,3537,124,78,1,0,991,128,256,180,...,81,1647,732,380,3309,973,82,0,26,5161


In [7]:
df_dropseq_HFiltNorm.head()

,1,2,3,4,5,6,7,8,9,10,...,3662,3663,3664,3665,3666,3667,3668,3669,3670,3671
H1-5,0,0,2,0,0,0,0,0,0,0,...,0,3,0,0,1,1,0,0,0,3
MALAT1,3,1,1,0,0,3,0,2,4,0,...,0,7,0,3,4,5,5,0,0,0
MT-RNR2,1,0,0,0,0,1,1,0,1,0,...,0,1,0,0,0,1,0,0,0,0
ARVCF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BCYRN1,2,1,1,0,0,0,1,1,3,0,...,0,0,0,4,0,0,0,2,3,0


We then take the transpose in order to have cells as rows and features as columns and then look at the structure.

In [8]:
df_smartseq_HFiltNormT = df_smartseq_HFiltNorm.T
df_smartseq_HFiltNormT.head()

,DDIT4,ANGPTL4,CALML5,KRT14,CCNB1,IGFBP3,AKR1C2,KRT6A,NDRG1,KRT4,...,MST1R,ZYG11A,NRG1,RBMS3,VCPIP1,LINC02693,OR8B9P,NEAT1,ZDHHC23,ODAD2
1,0,0,0,169,233,227,521,577,0,0,...,0,0,176,0,0,48,0,118,6,0
2,2475,0,0,0,3537,843,214,0,3,0,...,106,0,333,0,18,65,0,39,0,0
3,9088,2143,0,0,124,0,0,0,415,0,...,299,0,0,45,0,0,0,172,0,0
4,6909,3086,0,909,78,466,508,193,453,0,...,319,0,95,167,0,5,0,31,0,10
5,13655,2196,0,1,1,14688,3872,184,884,176,...,0,0,1,0,0,0,0,224,0,0


In [9]:
# we notice that the genes are in a different order than the training dataset, we reorder them based on the training dataset
columns_smartseq_file = f'../ai_lab/SmartSeq/{tumor}_SmartS_Filtered_Normalised_3000_Data_train.txt'
columns_smartseq_df = pd.read_csv(columns_smartseq_file, delimiter=' ')
columns_smartseq = columns_smartseq_df.T.columns
df_smartseq_HFiltNormT = df_smartseq_HFiltNormT[columns_smartseq]

In [10]:
df_smartseq_HFiltNormT.head()

,DDIT4,ANGPTL4,CALML5,KRT14,CCNB1,IGFBP3,AKR1C2,KRT6A,NDRG1,KRT4,...,MST1R,ZYG11A,NRG1,RBMS3,VCPIP1,LINC02693,OR8B9P,NEAT1,ZDHHC23,ODAD2
1,0,0,0,169,233,227,521,577,0,0,...,0,0,176,0,0,48,0,118,6,0
2,2475,0,0,0,3537,843,214,0,3,0,...,106,0,333,0,18,65,0,39,0,0
3,9088,2143,0,0,124,0,0,0,415,0,...,299,0,0,45,0,0,0,172,0,0
4,6909,3086,0,909,78,466,508,193,453,0,...,319,0,95,167,0,5,0,31,0,10
5,13655,2196,0,1,1,14688,3872,184,884,176,...,0,0,1,0,0,0,0,224,0,0


In [11]:
df_dropseq_HFiltNormT = df_dropseq_HFiltNorm.T
df_dropseq_HFiltNormT.head()

,H1-5,MALAT1,MT-RNR2,ARVCF,BCYRN1,ATXN7L2,IGFBP3,H1-3,CTIF,RNF123,...,BATF3,CDKN3,DLD,PMPCA,ZNF165,SCCPDH,NTAN1,CLIP2,DUSP23,ZNF682
1,0,3,1,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# we notice that the genes are in a different order than the training dataset, we reorder them based on the training dataset
columns_dropseq_file =  f'../ai_lab/DropSeq/{tumor}_Filtered_Normalised_3000_Data_train.txt'
columns_dropseq_df = pd.read_csv(columns_dropseq_file, delimiter=' ')
columns_dropseq = columns_dropseq_df.T.columns
df_dropseq_HFiltNormT = df_dropseq_HFiltNormT[columns_dropseq]
print(sum(df_dropseq_HFiltNormT.columns == columns_dropseq))

3000


In [13]:
df_dropseq_HFiltNormT.head()

,H1-5,MALAT1,MT-RNR2,ARVCF,BCYRN1,ATXN7L2,IGFBP3,H1-3,CTIF,RNF123,...,BATF3,CDKN3,DLD,PMPCA,ZNF165,SCCPDH,NTAN1,CLIP2,DUSP23,ZNF682
1,0,3,1,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Ensemble method

In [14]:
names = ['logreg', 'nn', 'xgboost']

In [15]:
class Ensemble():
    def __init__(self, models):
        self.models = models

    def predict(self, X):
        predictions = np.column_stack([model.predict(X) for model in self.models.values()])
        return np.mean(predictions, axis=1).astype(int)

    def score(self, X, y):
        return accuracy_score(y, self.predict(X))

In [16]:
models = {}
X = df_dropseq_HFiltNormT

for name in names:
    print(name)
    filename = f"pickle/{tumor}/{name}/dropseq_merged dataset"
    with open(filename, 'rb') as f:
        result = pickle.load(f)
        models[name] = result['model']


ensemble_model = Ensemble(models)

# Make predictions
ensemble_predictions = ensemble_model.predict(X)
prediction_dropseq_df = df_dropseq_HFiltNormT.copy()
prediction_dropseq_df['Prediction'] = ensemble_predictions
prediction_dropseq_df['Prediction'] = prediction_dropseq_df['Prediction'].apply(lambda x: 'Normoxia' if x == 1 else 'Hypoxia')
prediction_dropseq_df.head()

logreg
nn
xgboost


,H1-5,MALAT1,MT-RNR2,ARVCF,BCYRN1,ATXN7L2,IGFBP3,H1-3,CTIF,RNF123,...,CDKN3,DLD,PMPCA,ZNF165,SCCPDH,NTAN1,CLIP2,DUSP23,ZNF682,Prediction
1,0,3,1,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Normoxia
2,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Normoxia
3,2,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Normoxia
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Hypoxia
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Hypoxia


In [17]:
models = {}
X = df_smartseq_HFiltNormT

for name in names:
    print(name)
    filename = f"pickle/{tumor}/{name}/smartseq_merged dataset"
    with open(filename, 'rb') as f:
        result = pickle.load(f)
        models[name] = result['model']


ensemble_model = Ensemble(models)

# Make predictions
ensemble_predictions = ensemble_model.predict(X)
prediction_smartseq_df = df_smartseq_HFiltNormT.copy()
prediction_smartseq_df['Prediction'] = ensemble_predictions
prediction_smartseq_df['Prediction'] = prediction_smartseq_df['Prediction'].apply(lambda x: 'Normoxia' if x == 1 else 'Hypoxia')
prediction_smartseq_df.head()

logreg
nn
xgboost


,DDIT4,ANGPTL4,CALML5,KRT14,CCNB1,IGFBP3,AKR1C2,KRT6A,NDRG1,KRT4,...,ZYG11A,NRG1,RBMS3,VCPIP1,LINC02693,OR8B9P,NEAT1,ZDHHC23,ODAD2,Prediction
1,0,0,0,169,233,227,521,577,0,0,...,0,176,0,0,48,0,118,6,0,Normoxia
2,2475,0,0,0,3537,843,214,0,3,0,...,0,333,0,18,65,0,39,0,0,Normoxia
3,9088,2143,0,0,124,0,0,0,415,0,...,0,0,45,0,0,0,172,0,0,Hypoxia
4,6909,3086,0,909,78,466,508,193,453,0,...,0,95,167,0,5,0,31,0,10,Hypoxia
5,13655,2196,0,1,1,14688,3872,184,884,176,...,0,1,0,0,0,0,224,0,0,Hypoxia


In [18]:
# export the prediction dataframes to csv
prediction_dropseq_df.to_csv(f'../ai_lab/predictions/DropSeq{tumor}.csv')
prediction_smartseq_df.to_csv(f'../ai_lab/predictions/SmartSeq{tumor}.csv')